## Personal Project:Counterfactual evaluation with Causal-Design Recommendation Systems

### Introduction
 This is a project extension for David Rosenberg. 
 
 we're going to be reproducing a few results from http://proceedings.mlr.press/v97/vlassis19a.html, and extending their results in a few ways.  Here's an overview: We start by taking a multiclass classification problem and splitting it into train and test.  There are 26 classes, which we'll interpret as 26 possible actions to take for every input context. On the training set, we fit a multinomial logistic regression model to predict the correct label/best action.  Following the paper, we create a logging policy based on this model (details supplied in the relevant spot below).  We then generate "logged bandit feedback" for this logging policy using the **test** set.  Given this logged bandit feedback, we'll try out several different methods for estimating the value of various policies.  We'll also estimate the value of each of these policies using the full-feedback (i.e. the full observed rewards), and we'll treat that as the ground truth value for the purpose of performance assessment.

### Pre-setting

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import abc

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas.api.types import is_integer_dtype
import numpy as np
from numpy.random import default_rng
from scipy.special import expit
import seaborn as sns
import warnings;
warnings.filterwarnings('ignore');
import sys
%matplotlib widget

In [9]:
def get_fully_observed_bandit():
    """
    This loads in a multiclass classification problem and reformulates it as a fully observed bandit problem.
    
    """
    df_l = pd.read_csv('letter-recognition.data',
                       names = ['a']+[f'x{i}' for i in range(16)])
    X = df_l.drop(columns=['a'])

    # Convert labels to ints and one-hot
    y = df_l['a']
    # if y is not column of integers (that represent classes), then convert
    if not is_integer_dtype(y.dtype):
        y = y.astype('category').cat.codes

    ## Full rewards
    n = len(y)
    k = max(y)+1
    full_rewards = np.zeros([n, k])# return an array that follows the shape of[nrow,kcol]
    full_rewards[np.arange(0,n),y] = 1
    contexts = X
    best_actions = y
    return contexts, full_rewards, best_actions

In [10]:
#create a temporary view of Data.
df_1_c= pd.read_csv('letter-recognition.data',
                       names = ['a']+[f'x{i}' for i in range(16)]) # create the the ordered cols following the context index 1-16; using function"f"
df_1_c
yc = df_1_c['a'].astype('category').cat.codes
v_rewards = np.zeros([len(yc),max(yc)+1])
v_rewards[np.arange(0,len(yc)),yc]=1 # np.arange is to generate an array starting with a and stopping with b
v_rewards

    

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [11]:
contexts, full_rewards, best_actions = get_fully_observed_bandit()
n, k = full_rewards.shape
_, d = contexts.shape
print(f"There are {k} actions, the context space is {d} dimensional, and there are {n} examples.")
print(f"For example, the first item has context vector:\n{contexts.iloc[0:1]}.")
print(f"The best action is {best_actions[0]}.  The reward for that action is 1 and all other actions get reward 0.")
print(f"The reward information is store in full_rewards as the row\n{full_rewards[0]}.")
get_fully_observed_bandit()

There are 26 actions, the context space is 16 dimensional, and there are 20000 examples.
For example, the first item has context vector:
   x0  x1  x2  x3  x4  x5  x6  x7  x8  x9  x10  x11  x12  x13  x14  x15
0   2   8   3   5   1   8  13   0   6   6   10    8    0    8    0    8.
The best action is 19.  The reward for that action is 1 and all other actions get reward 0.
The reward information is store in full_rewards as the row
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0.].


(       x0  x1  x2  x3  x4  x5  x6  x7  x8  x9  x10  x11  x12  x13  x14  x15
 0       2   8   3   5   1   8  13   0   6   6   10    8    0    8    0    8
 1       5  12   3   7   2  10   5   5   4  13    3    9    2    8    4   10
 2       4  11   6   8   6  10   6   2   6  10    3    7    3    7    3    9
 3       7  11   6   6   3   5   9   4   6   4    4   10    6   10    2    8
 4       2   1   3   1   1   8   6   6   6   6    5    9    1    7    5   10
 ...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...
 19995   2   2   3   3   2   7   7   7   6   6    6    4    2    8    3    7
 19996   7  10   8   8   4   4   8   6   9  12    9   13    2    9    3    7
 19997   6   9   6   7   5   6  11   3   7  11    9    5    2   12    2    4
 19998   2   3   4   2   1   8   7   2   6  10    6    8    1    9    5    8
 19999   4   9   6   6   2   9   5   3   1   8    1    8    2    7    2    8
 
 [20000 rows x 16 columns],
 array([[0., 0., 0., ..., 0., 0., 0.],
       

In [6]:
## Choose train/test indices
rng = default_rng(7)
train_frac = 0.5
train_size = round(train_frac * n)
train_idx = rng.choice(n, size = train_size, replace = False)# this command is used to randomly select a fixed number of observations
test_idx = np.setdiff1d(np.arange(n), train_idx, assume_unique=True)

In [7]:
name_policy = [f"P{x}" for x in range(26)] #this is to generate similar column names
name_contexts = [f"Con{i}" for i in range(16)]
df_f = pd.DataFrame(np.concatenate((full_rewards,contexts),axis=1 ),columns=name_policy+name_contexts)
df_f['ActionTaken']=best_actions
df_f


,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,Con7,Con8,Con9,Con10,Con11,Con12,Con13,Con14,Con15,ActionTaken
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,6.0,10.0,8.0,0.0,8.0,0.0,8.0,19
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,4.0,13.0,3.0,9.0,2.0,8.0,4.0,10.0,8
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,6.0,10.0,3.0,7.0,3.0,7.0,3.0,9.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,6.0,4.0,4.0,10.0,6.0,10.0,2.0,8.0,13
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,6.0,6.0,6.0,5.0,9.0,1.0,7.0,5.0,10.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,6.0,6.0,6.0,4.0,2.0,8.0,3.0,7.0,3
19996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,9.0,12.0,9.0,13.0,2.0,9.0,3.0,7.0,2
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,7.0,11.0,9.0,5.0,2.0,12.0,2.0,4.0,19
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,6.0,10.0,6.0,8.0,1.0,9.0,5.0,8.0,18


In [8]:
type(np.array(contexts)[[2],[4]])

numpy.ndarray

In [9]:
df_m=(df_f.iloc[:,1:17].melt())
df_m[df_m["value"]==1].sort_index
psm1 = LogisticRegression(multi_class="multinomial")
fpsm = psm1.fit(contexts,best_actions)
ps=psm1.predict_proba(contexts)
#ps = pd.DataFrame(fpsm,columns=[f"a{i}" for i in range(16)])
ps_var = [f"A{r}" for r in range(26)]
ps_df = pd.DataFrame(data= ps,dtype=float, columns=ps_var)
ps_df = ps_df.round(6)
#a = [ps_df.iloc[[i],[v]]for i in ps_df.shape[0], for v in ps_df.shape(1) if ]
ps_df['ActionTaken']=best_actions
ps_df["Chosen"]=ps[range(best_actions.shape[0]),best_actions]
type(ps_df)
ps_df
ps.shape

(20000, 26)

(200,)

In [11]:
contexts.iloc[[1]]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10


In [12]:
r= np.array(full_rewards)
trial = [np.inner(r[[i]],ps[[i]]) for i in range(contexts.shape[0])]


In [13]:
np.mean(trial)

0.6200568740371194

In [14]:
a = np.array([rng.choice(26,p=p) for p in ps])
a

array([19, 16,  9, ..., 19, 18,  0])

In [17]:
#incorrect function for corresponding probability
def corres_prob(propMatrix,action):
    p= pd.DataFrame()
    n = propMatrix.shape[0]
    m = best_actions.shape[0]
    k = propMatrix.shape[1]
    for i,j in [[i,j]for i in range(n) for j in range(m)]:
            a=[propMatrix[[i],[x]] for x in range(k) if np.array(x) == action[[j],:]]
        
    p["cor_p"]=a
    return(p)
corres_prob(propMatrix=ps_df,action=best_actions)

        




In [74]:
a = [1/20]*20
a

[0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05,
 0.05]

### Policies
In this section, we'll build out a Policy class, some specific policies, and evaluate policies on full-feedback data.

**Class Writing for Policy Evaluation** We will complete the Policy class and the UniformActionPolicy classes below. Run the code provided to get an estimate of the value of the uniform action policy using the test set.  

In [23]:
class Policy:
    def __init__(self, num_actions=2):
        self.num_actions = num_actions

    @abc.abstractmethod
    def get_action_distribution(self, X):
        """   
        This method is intended to be overridden by each implementation of Policy.

        Args:
            X (pd.DataFrame): context

        Returns:
            2-dim numpy array with the same number of rows as X and self.num_actions columns. 
                Each rows gives the policy's probability distribution over actions conditioned on the context in the corresponding row of X
        """   
        raise NotImplementedError("Must override method")

    def get_action_propensities(self, X, actions):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of actions
            actions (np.array): actions taken, represented by integers, corresponding to rows of X

        Returns:
            1-dim numpy array of probabilities (same size as actions) for taking each action in its corresponding context
        """   
        
        prop = self.get_action_distribution(X)
        prop_chosen = prop[range(X.shape[0]),actions]
        return prop_chosen
        
        
 
    def select_actions(self, X,rng=default_rng(1)):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of actions and propensities returned

        Returns:
            actions (np.array): 1-dim numpy array of length equal to the number of rows of X.  Each entry is an integer indicating the action selected for the corresponding context in X. 
                The action is selected randomly according to the policy, conditional on the context specified in the appropriate row of X.
            propensities (np.array): 1-dim numpy array of length equal to the number of rows of X; gives the propensity for each action selected in actions

        """
        prop= self.get_action_distribution(X)
        action_chosen = np.array([rng.choice(26,p=p)for p in prop])
        prop_chosen = prop[range(action_chosen.shape[0]),action_chosen]
        return action_chosen, prop_chosen


        
        
        
    def get_value_estimate(self, X, full_rewards):
        """   
        Args:
            X (pd.DataFrame): contexts, rows correspond to entries of full_rewards
            full_rewards (np.array): 2-dim numpy array with the same number of rows as X and self.num_actions columns; 
                each row gives the rewards that would be received for each action for the context in the corresponding row of X.
                This would only be known in a full-feedback bandit, or estimated in a direct method

        Returns:
            scalar value giving the expected average reward received for playing the policy for contexts X and the given full_rewards

        """   
        
        prop = self.get_action_distribution(X)
        #unit_estimate = np.multiply(full_rewards[range(prop_chosen.shape[0]),action_chosen],prop_chosen)
        unit_estimate= [np.inner(np.array(full_rewards)[[i]],prop[[i]]) for i in range(full_rewards.shape[0])]
        reward_est = np.mean(unit_estimate)
        return reward_est

class UniformActionPolicy(Policy): 
    def __init__(self, num_actions=2):
        self.num_actions = num_actions

    def get_action_distribution(self, X):
        unit_prop = np.array([[1/self.num_actions]*self.num_actions])
        uniform_prop = unit_prop.repeat(X.shape[0],axis=0)
        
        return uniform_prop
        

In [50]:
uniform_test = UniformActionPolicy(num_actions=k)
unif_plc = uniform_test.get_action_distribution(X=contexts)
unif_plc
dic_index = [f"U{i}" for i in range(k)]
dic_fm = pd.DataFrame(data=unif_plc,columns=dic_index)
dic_fm

,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,...,U16,U17,U18,U19,U20,U21,U22,U23,U24,U25
0,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
1,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
2,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
3,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
4,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
19996,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
19997,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462
19998,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,...,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462,0.038462


## Data Splitting

In [24]:
X_train = contexts.iloc[train_idx].to_numpy()
y_train = best_actions.iloc[train_idx].to_numpy()
X_test = contexts.iloc[test_idx].to_numpy()
y_test = best_actions.iloc[test_idx].to_numpy()
full_rewards_test = full_rewards[test_idx]
uniform_policy = UniformActionPolicy(num_actions=k)

In [79]:
generic_policy=Policy(num_actions=k)
display_distribution = generic_policy.select_actions(X=X_test,actions=y_test)
display_distribution


array([19, 18,  6, ...,  5, 22, 19])

In [25]:
uniform_policy_value = uniform_policy.get_value_estimate(X=X_test, full_rewards=full_rewards_test)
print(f"The estimate of the value of the uniform action policy using the full-feedback test set is {uniform_policy_value}.")

The estimate of the value of the uniform action policy using the full-feedback test set is 0.03846153846153845.


**Problem 2.**  Complete the SKLearnPolicy class below and run the code that creates two policies and estimates their values using the full reward information in the test set.  You should find that the deterministic policy has a higher value than the stochastic policy.  Nevertheless, why might one choose to deploy the stochastic policy rather than the deterministic policy?

In [39]:
class SKLearnPolicy(Policy):
    """ 
    An SKLearnPolicy uses a scikit learn model to generate an action distribution.  If the SKLearnPolicy is built with is_deterministic=False, 
    then the action distribution for a context x should be whatever predict_proba for the model returns.  If is_deterministic=True, then all the probability mass 
    should be concentrated on whatever predict of the model returns.
    """
    def __init__(self, model, num_actions=2, is_deterministic=False):
        self.is_deterministic = is_deterministic
        self.num_actions = num_actions
        self.model = model

    def get_action_distribution(self, X):
        
        if (self.is_deterministic):
        
         propensity = model.predict_proba(X)
         return propensity
        
    
        else:
         pred_action = model.predict(X)
         blank = np.zeros([X.shape[0],self.num_actions])
         blank[range(X.shape[0]),pred_action]=1
         propensity = blank
         
         return propensity

        

    def select_actions(self, X, rng=default_rng(1)):
        """ You don't technically have to override this function.  If you just delete this function, the parent class Policy can handle it in a generic way
        However, if is_deterministic=True, then the action distribution for each context is trivial -- it always puts probability one for a 
        particular action and 0 for the others. And so "randomly" selecting an action according to this distribution using the code you write
        for select_actions in the parent class (Policy) is very inefficient.  You can just use model.predict to get the actions that will be 
        selected for each context.  That's the idea of the if statement."""
        
       
          
             
        if (self.is_deterministic):
          propensity = self.get_action_distribution(X)
          chosen_actions = np.array([rng.choice(26, p =p) for p in propensity])
          chosen_prop = propensity[range(chosen_actions.shape[0]),chosen_actions]
          
          return chosen_actions,chosen_prop
        
        else:
          chosen_actions = model.predict(X)
          chosen_prop = np.array([1]*X.shape[0]).T
          return chosen_actions,chosen_prop
        
        

model = LogisticRegression(multi_class='multinomial')
model.fit(X_train, y_train)
policy_stochastic = SKLearnPolicy(model=model, num_actions=k, is_deterministic=False)
policy_deterministic = SKLearnPolicy(model=model, num_actions=k, is_deterministic=True)

policy_stochastic_true_value = policy_stochastic.get_value_estimate(X_test, full_rewards_test)
policy_deterministic_true_value = policy_deterministic.get_value_estimate(X_test, full_rewards_test)
print(f"Stochastic policy true value {policy_stochastic_true_value}.")
print(f"Deterministic policy true value {policy_deterministic_true_value}.")

Stochastic policy true value 0.7631.
Deterministic policy true value 0.6261601176972126.


**Problem 3.** Fill in the VlassisLoggingPolicy class below, and evaluate the value of this logging policy using the code provided.

In [52]:
class VlassisLoggingPolicy(Policy):
    """
    This policy derives from another deterministic policy following the recipe described in the Vlassis et al paper, on the top of the second column in section 5.3.
    For any context x, if the deterministic policy selects action a, then this policy selects action a with probability eps (a supplied parameter), and spreads the
    rest of the probability mass uniformly over the other actions.
    """
    def __init__(self, deterministic_target_policy, num_actions=2, eps=0.05,):
        self.num_actions = num_actions
        self.target_policy = deterministic_target_policy
        self.eps = eps

    def get_action_distribution(self, X):
        blank = np.zeros([X.shape[0],self.num_actions])
        predicted_actions = model.predict(X)
        blank[range(X.shape[0]),predicted_actions]=self.eps
        blank[blank==0]=(1-self.eps)/25
        propensity = blank
        

        return propensity
        
    
logging_policy = VlassisLoggingPolicy(policy_deterministic, num_actions=k, eps=0.05)
logging_policy_value = logging_policy.get_value_estimate(X=X_test, full_rewards=full_rewards_test)
print(f"The estimate of the value of the logging policy using the full-feedback test set is {logging_policy_value}.")

The estimate of the value of the logging policy using the full-feedback test set is 0.0471572.


In [137]:
test = VlassisLoggingPolicy(policy_deterministic,num_actions=k, eps=0.05)
a= test.get_action_distribution(X=X_test)
a = pd.DataFrame(a,columns=[f"A{i}" for i in range(26)])
a


,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A16,A17,A18,A19,A20,A21,A22,A23,A24,A25
0,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.050,0.038,0.038,0.038,0.038,0.038,0.038
1,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.050,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
2,0.038,0.038,0.038,0.038,0.038,0.038,0.050,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
3,0.038,0.050,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
4,0.050,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.038,0.038,0.038,0.038,0.038,0.050,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
9996,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
9997,0.038,0.038,0.038,0.038,0.038,0.050,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038
9998,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,0.038,...,0.038,0.038,0.038,0.038,0.038,0.038,0.050,0.038,0.038,0.038


### Simulate bandit feedback and on-policy evaluation
**Problem 4.** Take a look at the generate_bandit_feedback function, so you understand how it works.  Then generate bandit feedback using the test data -- generate as many rounds are there are contexts in the test data. Use the result to generate an "on-policy" estimate of the value of the logging policy.  How does it compare to our "ground truth" estimate you found previously using the full-feedback test set? Repeat using 1/100th, 1/10th, and 10x as much bandit feedback, to see how much the value estimates change.

In [27]:
def generate_bandit_feedback(contexts, full_rewards, policy,
                             new_n = None,
                             rng=default_rng(1)):
    """   
    Args:
        contexts (np.array): contexts, rows correspond to entries of rewards
        full_rewards (np.array): 2-dim numpy array with the same number of rows as X and number of columns corresponding to the number actions
            each row gives the reward that would be received for each action for the context in the corresponding row of X. 

    Returns:
        new_contexts (np.array): new_n rows and same number of columns as in contexts
        actions (np.array): vector with new_n entries giving actions selected by the provided policy for the contexts in new_contexts
        observed_rewards (np.array): vector with new_n entries giving rewards received for the actions taken (in actions) in each context of new_contexts 
    """   
    
    if new_n is None:
        new_n = contexts.shape[0]
    n, k = full_rewards.shape
    num_repeats = np.ceil(new_n / n).astype(int)
    new_contexts = np.tile(contexts, [num_repeats,1])
    new_contexts = new_contexts[0:new_n]
    new_rewards = np.tile(full_rewards, [num_repeats,1])
    new_rewards = new_rewards[0:new_n]
    actions, propensities = policy.select_actions(X=new_contexts, rng=rng)
    observed_rewards = new_rewards[np.arange(new_n), actions]
    return new_contexts, actions, observed_rewards, propensities

    

In [42]:
a,b,c,d=generate_bandit_feedback(contexts = X_test,full_rewards=full_rewards_test,policy=policy_deterministic, new_n =None ,rng=default_rng(2))
View = pd.DataFrame(data=a, columns=[f"X{i}" for i in range(16)])
View["Action Observed"]=b
View["Reward Observed"]=c
View["Propensity"] = d
View


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,Action Observed,Reward Observed,Propensity
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8,19,1.0,0.960960
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10,8,1.0,0.447682
2,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10,16,0.0,0.232234
3,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10,1,1.0,0.819207
4,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7,0,1.0,0.839590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6,12,6,7,3,6,8,3,6,13,7,7,2,9,3,7,5,0.0,0.569896
9996,7,10,5,5,2,6,11,5,4,11,9,4,4,11,3,10,15,0.0,0.561289
9997,2,1,3,2,1,4,10,3,5,10,8,5,0,9,3,7,5,0.0,0.876559
9998,3,8,5,6,5,11,11,2,2,5,8,7,7,12,1,7,22,1.0,0.958187


### Test out off-policy value estimators
**This is the final research question and the foundation of more advanced methods for Causal-based Recommendation Systems targeting Counterfactual Learning** 

we will complete the get_value_estimators function below, per the specification.  Include the following estimators
- Unweighted mean (**matching the complete estimator in counterfactuals**)
- Importance-weighted (IW) value estimator(**MAR Situation**)
- Self-normalized IW mean(**Reducing the variance of estimator in MAR missing data setting**)
- Direct method with linear ridge regression reward predictor fit for each action
- Direct method with IW-linear ridge regression reward predictor fit for each action
- [**Testing for properties of this estimator**] Direct method with a non-linear reward predictor fit for each action
- [**think: Does Causal Forest work well/Recursive Partitioning**] Direct method with a non-linear reward predictor fit for all actions at once (action becomes part of the input)

**Our New Explorations**

- Multiple-action Causal Forest to estimate the counterfactuals(2019,2021 papers)
- Sufficient statistics with Covariate Shift Sensivisity Value(This is the idea from a paper proposed by Yuan 2018:sensitivity value)
- ...

Run the code below that will apply your value estimators to a policy on logged bandit feedback. 

In [ ]:
class TriageEstimator(Policy):
    def __init__(self, num_actions=2, num_estimators=2, est_indicator=[]):
     self.num_actions=num_actions
     self.num_estimators = num_estimators
     self.est_assignment = est_indicator
    @abc.abstractmethod
    
    def CompleteCase(X,actions, full_rewards ):
       


In [ ]:
## Build our value estimators

def get_value_estimators(policy, contexts, actions, rewards, propensities, skip_slow_stuff=False):
    """   
    Args:
        policy (Policy): the policy we want to get a value estimate for
        contexts (np.array): contexts from bandit feedback
        actions (np.array): actions chosen for bandit feedback
        rewards (np.array): rewards received in bandit feedback
        propensities (np.array): the propensity for each action selected under the logging policy (which is not provided to this function)
        skip_slow_stuff (boolean): boolean flag which allows you to turn on/off some slow estimators (ignore this if you like)
    Returns:
        est (dict): keys are string describing the value estimator, values are the corresponding value estimates 
    """   
    ## For IW estimator:
    est = {}
    #IW estimator:
    imp_w = 
    est["mean"] = np.mean(rewards)
    
    
    ## TODO

    return est


In [ ]:
def get_estimator_stats(estimates, true_parameter_value=None):
    """
 
     Args:
        estimates (pd.DataFrame): each row corresponds to collection of estimates for a sample and
            each column corresponds to an estimator
        true_parameter_value (float): the true parameter value that we will be comparing estimates to
            
    Returns:
        pd.Dataframe where each row represents data about a single estimator
    """
    
    est_stat = []
    for est in estimates.columns:
        pred_means = estimates[est]
        stat = {}
        stat['stat'] = est
        stat['mean'] = np.mean(pred_means)
        stat['SD'] = np.std(pred_means)
        stat['SE'] = np.std(pred_means) / np.sqrt(len(pred_means))
        if true_parameter_value:
            stat['bias'] = stat['mean'] - true_parameter_value
            stat['RMSE'] = np.sqrt(np.mean((pred_means - true_parameter_value) ** 2))
        est_stat.append(stat)

    return pd.DataFrame(est_stat)

In [ ]:
contexts_test, actions_test, rewards_test, propensities_test = generate_bandit_feedback(contexts=X_test, full_rewards=full_rewards_test, policy=logging_policy, rng=default_rng(6))
policy = policy_deterministic
est = get_value_estimators(policy, contexts_test, actions_test, rewards_test, propensities_test)
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
print(f"policy true value {policy_true_value}.")
df = pd.DataFrame(est, index=[0])
est

**Problem 6.** Run the code below to test your value estimators across multiple trials.  Write a few sentences about anything you learned from these experiments or that you find interesting.

In [ ]:
trials=20
val_ests = []
policy = policy_deterministic
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))

In [ ]:
trials=20
val_ests = []
policy = policy_stochastic
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))

In [ ]:
trials=20
val_ests = []
policy = uniform_policy
policy_true_value = policy.get_value_estimate(X_test, full_rewards_test)
rng=default_rng(6)
for i in range(trials):
    contexts, actions, rewards, propensities = generate_bandit_feedback(X_test, full_rewards_test, logging_policy, rng=rng)
    est = get_value_estimators(policy, contexts, actions, rewards, propensities)
    val_ests.append(est)

df = pd.DataFrame(val_ests)
print(get_estimator_stats(df, true_parameter_value=policy_true_value))